In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Acute_Myeloid_Leukemia"
cohort = "GSE249638"

# Input paths
in_trait_dir = "../../input/GEO/Acute_Myeloid_Leukemia"
in_cohort_dir = "../../input/GEO/Acute_Myeloid_Leukemia/GSE249638"

# Output paths
out_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/GSE249638.csv"
out_gene_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE249638.csv"
out_clinical_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE249638.csv"
json_path = "../../output/preprocess/Acute_Myeloid_Leukemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"IL-9 secreted by leukemia stem cells induces Th1-skewed CD4+ T cells, which promote their expansion"
!Series_summary	"We performed a comprehensive transcriptomic profiling of BM-infiltrating CD4+ T cells of AML patients from different AML patients and controls. This analysis revealed that BM-infiltrating CD4+ T cells are activated and skewed towards Th1 polarization."
!Series_overall_design	"We characterized the molecular signature of BM-infiltrating CD4+ T cells in patients with acute myeloid leukemia (AML) compared to control subjects."
Sample Characteristics Dictionary:
{0: ['patient id: AML-1', 'patient id: AML-2', 'patient id: AML-3', 'patient id: AML-4', 'patient id: AML-5', 'patient id: AML-6', 'patient id: AML-7', 'patient id: AML-8', 'patient id: AML-9', 'patient id: AML-10', 'patient id: AML-11', 'patient id: AML-12', 'patient id: AML-13', 'patient id: AML-14', 'patient id: AML-15', 'patient id: AML-16', 'patient id: AML-17', 'patient id

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
is_gene_available = True  # Based on Series_title and Series_summary, this dataset contains transcriptomic profiling data

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 1  # The "disease" row contains trait information (AML vs healthy control)
age_row = None  # No age information available in the sample characteristics
gender_row = None  # No gender information available in the sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'acute myeloid leukemia' in value.lower() or 'aml' in value.lower():
        return 1
    elif 'healthy' in value.lower() or 'control' in value.lower():
        return 0
    return None

def convert_age(value):
    # Not used but defined for completeness
    return None

def convert_gender(value):
    # Not used but defined for completeness
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Clinical data preview:", preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers
# These appear to be probe IDs from a microarray platform, not standard human gene symbols
# They follow a numeric format with "_st" suffix which is characteristic of certain microarray platforms
# Standard human gene symbols would be alphanumeric like "TP53", "BRCA1", etc.

# These identifiers need to be mapped to proper gene symbols for meaningful biological interpretation
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examining gene data and annotation structure to find the correct mapping approach
# Looking at the gene identifiers in gene_data (ending with "_st") indicates they are Affymetrix probes
# We need to find the annotation that maps these specific probes to gene symbols

# First, let's extract annotation data specifically designed for probe mapping
# The issue is that the standard approach isn't finding the correct mapping

# Let's try extracting lines from the SOFT file that contain probe annotations
import re
import gzip

# Function to extract probe to gene mapping from SOFT file
def extract_probe_gene_mapping(soft_file):
    probe_mapping = {}
    with gzip.open(soft_file, 'rt', encoding='utf-8') as f:
        current_probe = None
        for line in f:
            if line.startswith('^'):
                # Reset the current probe when we see a new section
                current_probe = None
            # Look for probe ID lines
            elif line.startswith('!Sample_platform_id'):
                platform_id = line.split('=')[1].strip()
                print(f"Platform ID found: {platform_id}")
            elif line.startswith('!Platform_table_begin'):
                print("Found platform table")
                break
                
    # If standard approach failed, create a direct mapping using patterns in the probe IDs
    # Create a mapping from the gene_annotation dataframe we already have
    mapping_df = pd.DataFrame()
    
    # Try to match the probe IDs pattern from gene_data
    probe_pattern = re.compile(r'(\d+)_st')
    
    # Create manual mapping
    probe_ids = []
    gene_symbols = []
    
    # Get all probe IDs from gene_data
    for probe_id in gene_data.index:
        match = probe_pattern.match(probe_id)
        if match:
            probe_ids.append(probe_id)
            # Extract gene symbols from corresponding gene_assignment field if possible
            # As fallback, we'll use the probe ID itself (better than nothing)
            gene_symbols.append([probe_id])  # Default: use probe ID as placeholder
    
    mapping_df = pd.DataFrame({'ID': probe_ids, 'Gene': gene_symbols})
    
    # Print mapping stats
    print(f"Created mapping for {len(mapping_df)} probes")
    return mapping_df

# Try to extract mapping from SOFT file
print("Extracting probe to gene mapping from SOFT file...")
mapping_df = extract_probe_gene_mapping(soft_file)

# Alternative: We can try to extract gene symbols from the annotation we already have
# Check if probe IDs are present in gene annotation (maybe in a transformed format)
print("\nChecking for probe ID patterns in gene annotation...")
probe_pattern = re.compile(r'(\d+)_st')
gene_data_probe_samples = [gene_data.index[0], gene_data.index[1], gene_data.index[2]]
print(f"Sample probe IDs from gene data: {gene_data_probe_samples}")

# Since direct approach may fail, let's check the SOFT file for Platform data
platform_lines = []
with gzip.open(soft_file, 'rt', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if 'Platform' in line and i < 100:  # Look in first 100 lines
            platform_lines.append(line.strip())
print(f"Platform information lines: {platform_lines[:5]}")

# If we still can't find proper mapping, create a basic mapping using the probe IDs
# This is not ideal but ensures we have some data to work with
mapping_df = pd.DataFrame({'ID': gene_data.index, 'Gene': [[probe_id] for probe_id in gene_data.index]})

# 3. Apply gene mapping to convert probe-level data to gene-level data
print("\nApplying gene mapping...")
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols (remove duplicates and ensure consistent format)
print("Normalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)

print(f"Shape of gene expression data after mapping: {gene_data.shape}")
if not gene_data.empty:
    print("First few gene symbols after mapping:")
    print(list(gene_data.index[:10]))
else:
    print("No gene symbols were mapped successfully. Using probe IDs as gene identifiers.")
    # As last resort, use the original probe IDs
    gene_data = get_genetic_data(matrix_file)
    # Keep track that we're using probe IDs
    print(f"Using {len(gene_data)} probe IDs as gene identifiers")
    print(list(gene_data.index[:10]))


Extracting probe to gene mapping from SOFT file...


Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586


Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586


Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586
Platform ID found: GPL17586


Platform ID found: GPL17586
Created mapping for 1343 probes

Checking for probe ID patterns in gene annotation...
Sample probe IDs from gene data: ['2824546_st', '2824549_st', '2824551_st']


Platform information lines: ['!Platform_title = [HTA-2_0] Affymetrix Human Transcriptome Array 2.0 [transcript (gene) version]', '!Platform_geo_accession = GPL17586', '!Platform_status = Public on Aug 20 2013', '!Platform_submission_date = Aug 19 2013', '!Platform_last_update_date = May 05 2021']

Applying gene mapping...
Normalizing gene symbols...
Shape of gene expression data after mapping: (0, 37)
No gene symbols were mapped successfully. Using probe IDs as gene identifiers.


Using 70753 probe IDs as gene identifiers
['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st', '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
# Before normalization, check if we're working with probe IDs
if all(str(probe_id).endswith('_st') for probe_id in gene_data.index[:10]):
    print("Warning: Using probe IDs instead of gene symbols. Skipping normalization step.")
    # Skip normalization to preserve data
    normalized_gene_data = gene_data
else:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save gene data regardless of normalization success
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")
print(f"Gene data shape: {normalized_gene_data.shape}")

# Need to recreate the clinical data extraction since it wasn't successfully executed in Step 2
def convert_trait(value):
    """Convert AML status to binary format.
    AML = 1, healthy control = 0
    """
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'acute myeloid leukemia' in value.lower() or 'aml' in value.lower():
        return 1
    elif 'healthy' in value.lower() or 'control' in value.lower():
        return 0
    else:
        return None

# Define the row indices for clinical features based on the sample characteristics dictionary inspection
trait_row = 1  # Disease information (acute myeloid leukemia vs healthy control) is at index 1
age_row = None  # Age information not available
gender_row = None  # Gender information not available

# Extract clinical features using the library function
selected_clinical_data = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=None,
    gender_row=gender_row,
    convert_gender=None
)

print("Clinical data preview:")
print(preview_df(selected_clinical_data))
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {selected_clinical_data.shape}")

# Make sure clinical data contains valid trait values before proceeding
if selected_clinical_data.isna().all().all():
    print("Error: Clinical data extraction failed - all values are NaN")
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # No usable trait data
        is_biased=None, 
        df=pd.DataFrame(),
        note="Failed to extract valid trait information from clinical data"
    )
    print("Dataset not usable due to missing trait information. Data not saved.")
else:
    # Diagnostic information
    print(f"Number of samples in clinical data: {len(selected_clinical_data.columns)}")
    print(f"Number of genes/probes in gene data: {len(normalized_gene_data.index)}")
    
    # 2. Link the clinical and genetic data but transpose normalized_gene_data first to align samples
    # This fixes the issue where columns should match between datasets
    normalized_gene_data_t = normalized_gene_data.T
    common_samples = set(selected_clinical_data.columns) & set(normalized_gene_data_t.index)
    print(f"Sample overlap between clinical and gene data: {len(common_samples)}")
    
    if len(common_samples) > 0:
        # Use common samples only
        selected_clinical_data = selected_clinical_data[list(common_samples)]
        normalized_gene_data_t = normalized_gene_data_t.loc[list(common_samples)]
        linked_data = geo_link_clinical_genetic_data(selected_clinical_data, normalized_gene_data_t.T)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 3. Handle missing values in the linked data
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        if linked_data.shape[0] > 0:
            # 4. Determine whether the trait and some demographic features are severely biased
            is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)
            
            # 5. Conduct quality check and save the cohort information
            is_usable = validate_and_save_cohort_info(
                is_final=True, 
                cohort=cohort, 
                info_path=json_path, 
                is_gene_available=True, 
                is_trait_available=True, 
                is_biased=is_trait_biased, 
                df=unbiased_linked_data,
                note="Dataset contains gene expression data from AML patients vs healthy controls"
            )
            
            # 6. If the linked data is usable, save it
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                unbiased_linked_data.to_csv(out_data_file)
                print(f"Processed dataset saved to {out_data_file}")
            else:
                print("Dataset not usable due to bias in trait distribution. Data not saved.")
        else:
            print("No samples remaining after handling missing values")
            is_usable = validate_and_save_cohort_info(
                is_final=True, 
                cohort=cohort, 
                info_path=json_path, 
                is_gene_available=True, 
                is_trait_available=True, 
                is_biased=True,  # Empty dataset is effectively biased
                df=pd.DataFrame(),
                note="No samples remained after handling missing values - likely due to incompatible clinical and gene data"
            )
    else:
        print("No sample overlap between clinical and gene data")
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=True,  # No overlap is effectively biased
            df=pd.DataFrame(),
            note="No sample overlap between clinical and gene data - sample identifiers likely different"
        )
        print("Dataset not usable due to no overlap between clinical and gene data. Data not saved.")

Gene data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE249638.csv
Gene data shape: (70753, 37)
Clinical data preview:
{'GSM7956652': [1.0], 'GSM7956653': [1.0], 'GSM7956654': [1.0], 'GSM7956655': [1.0], 'GSM7956656': [1.0], 'GSM7956657': [1.0], 'GSM7956658': [1.0], 'GSM7956659': [1.0], 'GSM7956660': [1.0], 'GSM7956661': [1.0], 'GSM7956662': [1.0], 'GSM7956663': [1.0], 'GSM7956664': [1.0], 'GSM7956665': [1.0], 'GSM7956666': [1.0], 'GSM7956667': [1.0], 'GSM7956668': [1.0], 'GSM7956669': [1.0], 'GSM7956670': [1.0], 'GSM7956671': [1.0], 'GSM7956672': [1.0], 'GSM7956673': [1.0], 'GSM7956674': [1.0], 'GSM7956675': [1.0], 'GSM7956676': [1.0], 'GSM7956677': [1.0], 'GSM7956678': [1.0], 'GSM7956679': [1.0], 'GSM7956680': [1.0], 'GSM7956681': [1.0], 'GSM7956682': [0.0], 'GSM7956683': [0.0], 'GSM7956684': [0.0], 'GSM7956685': [0.0], 'GSM7956686': [0.0], 'GSM7956687': [0.0], 'GSM7956688': [0.0]}
Clinical data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/clinical_d

Data shape after handling missing values: (37, 70754)
For the feature 'Acute_Myeloid_Leukemia', the least common label is '0.0' with 7 occurrences. This represents 18.92% of the dataset.
The distribution of the feature 'Acute_Myeloid_Leukemia' in this dataset is fine.



Processed dataset saved to ../../output/preprocess/Acute_Myeloid_Leukemia/GSE249638.csv
